# Dante

Following code is heavily inspired by these projects:
- https://github.com/mathematiguy/keras-char-rnn
- http://karpathy.github.io/2015/05/21/rnn-effectiveness/
- https://www.kaggle.com/mrisdal/intro-to-lstms-w-keras-gpu-for-text-generation/notebook

In [1]:
import numpy as np
import os
import sys
import random
import slabikar
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, LSTM
from keras.callbacks import ModelCheckpoint, LambdaCallback
from keras.utils import np_utils
from keras import backend as K
import keract

Using TensorFlow backend.


Hyperparameters are set here

In [2]:
maxlen = 120 #length of sequence
step = 13 #overlap
validation_split = 0.1
batch_size = 128
rnn_size = 1#128
num_layers = 2
drop_prob = 0.1
epochs = 6
temperature=1.0
sample_length = 1000

Function for concatenating all text files from directory. Text files are expected to be utf-8 encoded.

In [3]:
text_data = ''
for filename in filter(lambda s: s.endswith(".txt"), os.listdir('resources/')):
    # open file with default encoding
    print("loading file: %s" % filename)
    filepath = os.path.join('resources/', filename)
    with open(filepath,'r', encoding='utf-8') as f:
        text_data += f.read() + "\n"


loading file: Peklo.txt
loading file: Nebo.txt
loading file: Ocistec.txt


Since our dataset is pretty small, we convert everything to lowercase and remove diacritics. There are some characters that need to be tweaked beforehand.

In [4]:
text_data = text_data.replace("’","\'")
text_data = text_data.replace("„","\"")
text_data = text_data.replace("“","\"")
text_data = text_data.replace("‒","-")
import unidecode
text_data = unidecode.unidecode(text_data).lower()


Methods for processing texts. One uses syllables as text atoms, the other uses characters

Return values:
- atom_to_int: (dict) Maps characters in the character set to ints.
- int_to_atom: (dict) Maps ints to characters in the character set.
- n_atom: (int) The number of characters in the text.
- n_vocab: (int) The number of unique characters in the text.'''
- data: preprocessed input

In [5]:
def process_text_char(text_data):
    # create mapping of unique chars to integers, and a reverse mapping
    chars = sorted(set(text_data)) #sorted is necessary for checkpointing model 
    char_to_int = {c: i for i, c in enumerate(chars)}
    int_to_char = {i: c for i, c in enumerate(chars)}
    # summarize the loaded data
    n_chars = len(text_data)
    n_vocab = len(chars)    
    return char_to_int, int_to_char, n_chars, n_vocab, text_data

def process_text_syllable(text_data):
    syllable_data = slabikar.slabikar(text_data)
    syllables = sorted(set(syllable_data))
    syllable_to_int = {c: i for i, c in enumerate(syllables)}
    int_to_syllable = {i: c for i, c in enumerate(syllables)}
    # summarize the loaded data
    n_syllables = len(syllable_data)
    n_vocab = len(syllables)    
    return syllable_to_int, int_to_syllable, n_syllables, n_vocab, syllable_data

Processes data to overlapping sequences. Targets are single atoms. Syllable data cannot be preprocessed as whole. Use generator instead.

In [6]:
def createInput(text, maxlen, step, n_vocab, atom_to_int):
    dataX = []
    dataY = []
    for i in range(0, len(text) - maxlen - 1, step):
        seq_in = text[i: i + maxlen]
        seq_out = text[i+maxlen] #(text[i + 1: i + maxlen + 1]) #weird
        dataX.append([atom_to_int[atom] for atom in seq_in])
        dataY.append(atom_to_int[seq_out])
    #should one hot encode
    print(len(dataX))
    print(len(dataY))
    print(len(dataX[0]))
    X = np_utils.to_categorical(dataX, num_classes=n_vocab)
    y = np_utils.to_categorical(dataY, num_classes=n_vocab)
    #same thing as
    #X = np.zeros((len(sentences), maxlen, n_vocab), dtype=np.bool)
    #y = np.zeros((len(sentences), n_vocab), dtype=np.bool)
    #for i in range(len(sentences)):
    #    sentence = sentences[i]
    #    target = targets[i]
    #    for j in range(maxlen):
    #        X[i][j][atom_to_int[sentence[j]]] = 1
    #    y[i][atom_to_int[target[j]]] = 1
    return X,y

def get_batch(batch, starts, text_data, seq_length, batch_size, 
              atom_to_int, n_vocab):
    dataX = []
    dataY = []
    for start in range(batch_size * batch, batch_size * (batch + 1)): 
        seq_in  = text_data[starts[start]:starts[start] + seq_length]
        seq_out = text_data[starts[start] + seq_length]
        dataX.append([atom_to_int[atom] for atom in seq_in])
        dataY.append(atom_to_int[seq_out])
        
    X = np_utils.to_categorical(dataX, num_classes=n_vocab)
    X = X.reshape(batch_size, seq_length, n_vocab) #necessary<
    y = np_utils.to_categorical(dataY, num_classes=n_vocab)
    return X, y

#use this for syllables
def generate_batches(mode, text_data, seq_length, validation_split,
                     batch_size, atom_to_int, n_atom, n_vocab,
                     random_seed=42, shuffle=True):#rovnaky seed zabezpeci rovnakost pre oba gneratory
    random.seed(random_seed)
    # index the text_data
    starts = list(range(n_atom - n_atom % seq_length - seq_length))
    if shuffle:#internal state?
        # shuffle the indices
        random.shuffle(starts)
    n_batches = n_atom // batch_size
    validation_size = round(n_batches * validation_split)
    while True:
        if mode == 'validation':
            for batch in range(validation_size):
                X, y = get_batch(batch, starts, text_data, seq_length, 
                                 batch_size, atom_to_int, n_vocab)
                yield X, y
            
        elif mode == 'train':
            for batch in range(validation_size, n_batches-1):
                X, y = get_batch(batch, starts, text_data, seq_length, 
                                 batch_size, atom_to_int, n_vocab)
                yield X, y
        else:
            raise ValueError("only 'validation' and 'train' modes accepted")


Model builder.

In [7]:
def build_model(batch_size, seq_length, n_vocab, rnn_size, num_layers, drop_prob):
    model = Sequential()
    for i in range(num_layers):
        if i == num_layers - 1:
            # add last hidden layer
            model.add(LSTM(rnn_size, return_sequences=False))
            #model.add(TimeDistributed(Dense(num_chars))) #what is better?
        elif i == 0:
            # add first hidden layer
            model.add(LSTM(rnn_size, batch_input_shape=(None, seq_length, n_vocab), return_sequences=True))
        else:
            # add middle hidden layer
            model.add(LSTM(rnn_size, return_sequences=True))
        
        model.add(Dropout(drop_prob))
    # add output layer
    model.add(Dense(n_vocab, activation='softmax'))
    # compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])  
    return model


Callback for printing while training

In [11]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def on_epoch_end(epoch, logs):
    # Function invoked for specified epochs. Prints generated text.
    # Using epoch+1 to be consistent with the training epochs printed by Keras
    if epoch % 5 == 0:
        print()
        print('----- Generating text after Epoch: %d' % epoch)
        #dirty global hack
        start_index = random.randint(0, n_atoms - maxlen - 1)
        for diversity in [0.2, 0.5, 1.0, 1.2]:
            print('----- diversity:', diversity)

            generated = ''
            sentence = data[start_index: start_index + maxlen]
            generated += sentence
            print('----- Generating with seed: "' + sentence + '"')
            sys.stdout.write(generated)

            for i in range(400):
                x_pred = np.zeros((1, maxlen, n_vocab))
                for t, char in enumerate(sentence):
                    x_pred[0, t, atom_to_int[char]] = 1.

                preds = model.predict(x_pred, verbose=0)[0]
                next_index = sample(preds, diversity)
                next_char = int_to_atom[next_index]

                generated += next_char
                sentence = sentence[1:] + next_char

                sys.stdout.write(next_char)
                sys.stdout.flush()
            print()
    else:
        print()
        print('----- Not generating text after Epoch: %d' % epoch)

generate_text = LambdaCallback(on_epoch_end=on_epoch_end)

Use this to prepare char data (contains some debug printing to ensure everything looks good)

In [9]:
atom_to_int, int_to_atom, n_atoms, n_vocab, data = process_text_char(text_data)
X,y = createInput(data, maxlen, step, n_vocab, atom_to_int)
print(atom_to_int)
print(int_to_atom)
print(n_atoms)
print(n_vocab)
print(len(y))
print(len(X))

40438
40438
120
{'\n': 0, ' ': 1, '!': 2, '"': 3, "'": 4, '(': 5, ')': 6, ',': 7, '-': 8, '.': 9, ':': 10, ';': 11, '?': 12, 'a': 13, 'b': 14, 'c': 15, 'd': 16, 'e': 17, 'f': 18, 'g': 19, 'h': 20, 'i': 21, 'j': 22, 'k': 23, 'l': 24, 'm': 25, 'n': 26, 'o': 27, 'p': 28, 'q': 29, 'r': 30, 's': 31, 't': 32, 'u': 33, 'v': 34, 'w': 35, 'x': 36, 'y': 37, 'z': 38}
{0: '\n', 1: ' ', 2: '!', 3: '"', 4: "'", 5: '(', 6: ')', 7: ',', 8: '-', 9: '.', 10: ':', 11: ';', 12: '?', 13: 'a', 14: 'b', 15: 'c', 16: 'd', 17: 'e', 18: 'f', 19: 'g', 20: 'h', 21: 'i', 22: 'j', 23: 'k', 24: 'l', 25: 'm', 26: 'n', 27: 'o', 28: 'p', 29: 'q', 30: 'r', 31: 's', 32: 't', 33: 'u', 34: 'v', 35: 'w', 36: 'x', 37: 'y', 38: 'z'}
525809
39
40438
40438


Train char model

In [ ]:
callbacks = [generate_text,ModelCheckpoint('checkpoints/weights-{epoch:02d}-{val_acc:.2f}-{val_loss:.2f}.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='max')]
model = build_model(batch_size, maxlen, n_vocab, rnn_size, num_layers, drop_prob)
model.fit(X,y,batch_size=batch_size,epochs=epochs,callbacks=callbacks, validation_split=validation_split)

Train on 36394 samples, validate on 4044 samples
Epoch 1/6
27904/36394 [======================>.......] - ETA: 23s - loss: 3.5314 - acc: 0.1513

Use this to prepare syllable model

In [ ]:
atom_to_int, int_to_atom, n_atoms, n_vocab, data = process_text_syllable(text_data)
#print(atom_to_int)
#print(int_to_atom)
print(n_atoms)
print(n_vocab)

Train syllable model

In [ ]:
callbacks = [generate_text,ModelCheckpoint('checkpoints/weights-{epoch:02d}-{val_acc:.2f}-{val_loss:.2f}.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='max')]
model = build_model(batch_size, maxlen, n_vocab, rnn_size, num_layers, drop_prob)

n_batches = len(data) // batch_size
batch_params = (data, maxlen, validation_split, batch_size, atom_to_int, n_atoms, n_vocab)
model.fit_generator(
    generator = generate_batches('train', *batch_params),
    validation_data = generate_batches('validation', *batch_params),
    validation_steps = int(n_batches * validation_split),
    epochs = epochs,
    steps_per_epoch = n_batches-1,
    callbacks = callbacks)

Here we can load saved model to play around with it (make sure to use correct preprocessing for loaded model).

In [ ]:
model = load_model('.h5')#specify the file you want to use

Following function is used to generate some sample outputs from trained model. It also saves activations of neurons.

In [ ]:
def pick_atom_index(predictions, temperature)
    preds = np.asarray(predictions).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def sample_model(model, seed, sample_length, int_to_atom, n_vocab, temperature, num_layers):
    '''Prints out a sequence of text generated from the text data'''
    # data = {data:[{poem,neurons}...],neurons}
    feed_seq = seed #String of maxlen, or maxlen syllables
    print('initialisation:', ''.join(feed_seq))
    results = [].append(seed)
    activations = [0]*len(seed) #no activations for seed
    feed_seq = [atom_to_int[i] for i in feed_seq]
    feed_seq = np_utils.to_categorical(feed_seq, num_classes=n_vocab)
    
    for _ in range(sample_length):
        adict = get_activations(model,feed_seq)
        preds = []
        for k,v in adict.items():
            if k[:4] == 'lstm':#this is interesting
                
            if k[:5] == 'dense': 
                preds = v
        preds = #model.predict(feed_seq, batch_size=1)
        predicted = pick_atom_index(preds, temperature)
        results.append(int_to_atom[predicted])
        
        feed_seq = np.reshape(feed_seq, seq_length)
        feed_seq = list(feed_seq)[1:] + [np.argmax(next_char)]
    
    return {'poem': results, 'neurons': activations}

    
 start_index = random.randint(0, len(user) - maxlen - 1)
        for diversity in [0.2, 0.5, 1.0, 1.2]:
            print('----- diversity:', diversity)

            generated = ''
            sentence = user[start_index: start_index + maxlen]
            generated += sentence
            print('----- Generating with seed: "' + sentence + '"')
            sys.stdout.write(generated)

            for i in range(400):
                x_pred = np.zeros((1, maxlen, len(chars)))
                for t, char in enumerate(sentence):
                    x_pred[0, t, char_indices[char]] = 1.

                preds = model.predict(x_pred, verbose=0)[0]
                next_index = sample(preds, diversity)
                next_char = indices_char[next_index]

                generated += next_char
                sentence = sentence[1:] + next_char
    
sample_model(model, 25, n_vocab)